<a href="https://colab.research.google.com/github/edersonss1987/Project_person/blob/main/streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importe de pacotes necessarios
import pandas as pd
import streamlit as st
import numpy as np
import json
import requests
import time
import datetime
import timedelta
import pages as page
import altair as alt
import plotly.express as px
import plotly.figure_factory as ff
from PIL import Image
from streamlit_lottie import st_lottie
from streamlit_option_menu import option_menu
from io import StringIO

# Formatação da Pafina
st.set_page_config(
    layout='wide',
    page_title='App Eder',
    page_icon="🧊",
    initial_sidebar_state="auto"
)


# Cache da animação do botão clique aqui
@st.cache_data
def load_lottie(url):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()


# Botões do menu principal
selected = option_menu(None,
                       ["Home", "Upload"],
                       icons=["house", "upload"],
                       default_index=0,
                       orientation="horizontal"
                       )


# Se botão Upload else Home
if selected == "Upload":
    try:
        uploaded_file = st.file_uploader(
            "*Apenas são válidos arquivo.csv, separados com ( ; )*", type=['csv'])
        # função de upload do arquivo.csv

        @st.cache_data
        def upload_csv(experimental_allow_widgets=True):
            if uploaded_file is not None:
                file_details = {
                    'nome do arquivo': uploaded_file.name, 'tamanho': uploaded_file.size}
                st.write(file_details)
                dataframe = uploaded_file
                with st.spinner('Aguade o fim da Importação'):
                    time.sleep(2)
                st.success('Pronto!')

            return dataframe
        dataframe = upload_csv()

        st.write('''
            Erro ao carregar o *arquivo.csv*:
            Certifique-se que o separados está com ( ; ) e a quantidade de colunas não tenha excedido à 18''')

        st.write('''
            Selecione um *arquivo.csv''')

    except Exception:
        pass
elif selected == "Home":
    # Botão com texto de apresentação de analise
    st.write('*Saiba mais sobre a importância da Análise de Dados*')
    if st.button('clique aqui'):
        lottie_coding = load_lottie(
            'https://lottie.host/4c93e017-d624-4c06-bb82-04c3883f691e/KOhN7spfkK.json')
        lottie_coding_a = load_lottie(
            'https://lottie.host/eef1e455-c9f3-4a44-9ea8-943627f0d819/Sr4xGBvsfy.json')
        with st.container():
            st.write('---')
            left_column, right_column = st.columns(2)
            with left_column:
                st.header(' A importância da análise de dados ')
                st.write(
                    '''A análise de dados desempenha um papel fundamental
            na tomada de decisões eficazes e na obtenção de vantagem
            competitiva no mundo empresarial atual.
            À medida que o volume de dados disponíveis continua a crescer exponencialmente,
                a capacidade de compreender, interpretar e extrair insights
                valiosos desses dados tornou-se uma prioridade para as
                empresas que desejam prosperar,
            com tomada de decisões embasadas.'''
                )

                st.write(
                    '''A análise de dados fornece informações fundamentadas para a tomada de decisões.
            Com a devida análise, as empresas podem identificar padrões, tendências e
            correlações nos dados que ajudam a orientar escolhas estratégicas.
            Em vez de depender de suposições ou intuições,
            as decisões são baseadas em fatos tangíveis,
            reduzindo os riscos e aumentando as chances de sucesso.
            Identificação de oportunidades de negócios:'''
                )

                st.write(
                    '''Ao analisar grandes conjuntos de dados,
            as empresas podem identificar oportunidades de negócios antes dos concorrentes.
            Os insights derivados da análise podem revelar lacunas no mercado,
            necessidades dos clientes, preferências de compra e outras informações valiosas
            que permitem às empresas desenvolver produtos ou serviços inovadores e direcionar
            suas estratégias de marketing de forma mais eficaz.
            '''
                )

                with right_column:
                    st_lottie(lottie_coding, height=600, key='coding')
                    st.write(
                        '''Podemos tambem identificar melhoria da eficiência operacional,
                onde a análise de dados pode ajudar a otimizar processos internos,
                identificando ineficiências e áreas de melhoria.
                Ao analisar o desempenho operacional,
                as empresas podem identificar gargalos, reduzir custos,
                aprimorar a cadeia de suprimentos e otimizar a alocação de recursos.
                Isso resulta em uma maior eficiência e produtividade geral,
                melhorando a competitividade e o desempenho financeiro.'''
                    )

                    st.write(
                        '''Personalização e experiência do cliente: Com a análise de dados,
                as empresas podem entender melhor seus clientes e suas preferências.
                Ao analisar dados demográficos, comportamentais e históricos de compra,
                as empresas podem personalizar ofertas, recomendações e comunicações,
                criando uma experiência do cliente mais relevante e satisfatória.
                Isso não apenas aumenta a satisfação e a fidelidade do cliente,
                mas também impulsiona as vendas e o crescimento dos negócios.'''
                    )
                with left_column:
                    st_lottie(lottie_coding_a, height=580)

        if st.button('*menos*'):
            st.write('')
st.divider()


# função de arquivo do suporte
# carregamento rápido dos dados da função
try:
    @st.cache_data
    def arquivo_suporte():
        # colunas que seram aproveitadas na nossa analise
        col_names = ['Responsável', "Dt Abertura ", "Categoria", "Cliente",
                     "Desc Abreviada", "Status", "Dt Conclusão"]

        dados_suporte = pd.read_csv(
            dataframe, sep=';', encoding='utf-8', usecols=col_names)
        dados = pd.DataFrame(dados_suporte)

        # Setando a opção de visualizar todas as colunas
        pd.set_option('display.max_columns', None)
        # Fazendo a leitura do arquivo
        dados = dados_suporte.rename(columns={'Responsável': 'nome_analista', 'Dt Abertura ': 'data_da_abertura',
                                              'Categoria': 'produto', 'Desc Abreviada': 'tipo_do_chamado',
                                              'Cliente': 'nome_da_empresa', 'Status': 'status_do_chamado',
                                              'Dt Conclusão': 'data_da_conclusao'})

        # gerando um campo vazio na coluna de data, onde a mesma será excuída na analise
        erro_colunm = dados.loc[dados['data_da_conclusao'] == 'Resolvido']
        # st.write(f'LINHAS COM ERRO DE VALORES POR FAVOR CORRIJA\n\
        # PORTANTO ESSES DADOS SERÃO EXCLUÍDOS DA ANÁLISE\
        # {erro_colunm}'
        # )

        # excuindo os dados vaios gerados no erro de import nos campos de data
        dados = dados.dropna()

        # Alterando o tipo de dado das colunas para datas, pois as mesmas foram importadas como texto
        dados['data_da_abertura'] = pd.to_datetime(
            dados['data_da_abertura'], dayfirst=True)
        dados['data_da_conclusao'] = pd.to_datetime(
            dados['data_da_conclusao'], dayfirst=True)
        # dados['semana'] = dados['data_da_abertura'].dt.weekday
        # Criando uma chave com valores, para alteração da lingua conforme datas, meses e dias
        diaspt = {
            'Sunday': 'Domingo',
            'Monday': 'Segunda-feira',
            'Tuesday': 'Terça-feira',
            'Wednesday': 'Quarta-feira',
            'Thursday': 'Quinta-feira',
            'Friday': 'Sexta-feira',
            'Saturday': 'Sábado'
        }

        # semana = {
        #    '0': '1° Semana',
        #    'Monday': 'Segunda-feira',
        #    'Tuesday': 'Terça-feira',
        #    'Wednesday': 'Quarta-feira',
        #    'Thursday': 'Quinta-feira',
        #    'Friday': 'Sexta-feira',
        #    'Saturday': 'Sábado'
        # }

        mesespt = {
            'January': 'Janeiro',
            'February': 'Fevereiro',
            'March': 'Março',
            'April': 'Abril',
            'May': 'Maio',
            'June': 'Junho',
            'July': 'Julho',
            'August': 'Agosto',
            'September': 'Setembro',
            'October': 'Outubro',
            'November': 'Novembro',
            'December': 'Dezembro',
        }

        # Transformando dias numéricos, para dias da semana, propriamente dito, o mesmo processo se dá pelos meses.
        dados['dia_de_abertura'] = dados['data_da_abertura'].dt.day_name().replace(
            diaspt)
        dados['mes_de_abertura'] = dados['data_da_abertura'].dt.month_name().replace(
            mesespt)
        dados['ano_de_abertura'] = dados['data_da_abertura'].dt.strftime('%Y')
        dados['dia_de_fechamento'] = dados['data_da_conclusao'].dt.day_name().replace(
            diaspt)
        dados['mes_de_fechamento'] = dados['data_da_conclusao'].dt.month_name(
        ).replace(mesespt)
        dados['ano_de_fechamento'] = dados['data_da_conclusao'].dt.strftime(
            '%Y')
        # dados['tempo_de_chamado_aberto'] = dados['data_da_conclusao'] - dados['data_da_abertura']

        # Criando o valor de n_chamados com valor de "1", pois devemos somar valores nos tratamentos de dados, conforme linhas regristradas
        dados['n_chamados'] = 1
        dados['n_chamados'] = dados['n_chamados'].astype('int64')

        return dados
except NameError:
    st.write(NameError, 'Selecione um arquivo')


# Abertura do arquivo importado
try:
    dados = arquivo_suporte()
except NameError:
    pass
except UnicodeDecodeError:
    st.write('Erro: O codec utf-8 não pode decodificar o byte')


############################################# Configuração da barra lateral ########################################
with st.sidebar:
    st.image('logo.png', use_column_width=False)
    st.subheader(' ')
    menu_options = {
        'Suporte': ['Final ou Revenda', 'Suporte de Canais', 'Suporte Final'],
    }

    menu = st.selectbox('Menu', list(menu_options.keys()),
                        label_visibility='collapsed', disabled=True)
    if menu:
        submenu = st.selectbox(
            'Submenu', menu_options[menu], label_visibility='collapsed')
    st.header('Filtros')

    try:
        # analista_ = [' ', 'nicolas.costa','DavidPereira','henriquesilva','LucasMigalis','wesleyo,nelsons',\
        # 'andersonricardo','andre.holot','brayan','jorgeo','matheusmendes','TiagoVieira','lilian','LucasPereira',\
        # 'JhonataLopes','gabriel.tadeu','Gladson','vitor.santos','cristhiansoares','carlos.coimbra']

        opcao_analista = st.selectbox(
            'Nome do Analista',
            options=dados['nome_analista'].unique()
        )

        ############ Multiselect não funciona ########
        # opcao_analista = st.multiselect(            #
        #    'Nome do Analista',                     #
        #    options=dados['nome_analista'].unique(),#
        #    default= 'Gladson'                      #
        # )                                           #
        ##############################################

        ano_opcao = st.selectbox(
            'selecione o Ano',
            options=dados['ano_de_abertura'].unique()
        )

        mes_opcao = st.selectbox(
            'selecione o mês',
            options=dados['mes_de_abertura'].unique()
        )

        # produto = st.selectbox(
        #   'selecione o produto',
        #    options=dados['produto'].unique(), disabled=False
        # )

    except Exception:
        pass


############################################## Filtros da Barras Laterais ###########################################
try:
    # FILTRO 1 ----- filtra dados agrupados de analista, mes e ano
    chamados_total = dados.loc[
        (dados['nome_analista'] == opcao_analista) &
        (dados['mes_de_abertura'] == mes_opcao) &
        (dados['ano_de_abertura'] == ano_opcao)
    ]
    # _____________________________________________________________________________________________________________________

    # FILTRO 2 ----------- filtra mes e ano
    filto_ano_mes = dados.loc[
        (dados['ano_de_abertura'] == ano_opcao) &
        (dados['mes_de_abertura'] == mes_opcao)]
    # _____________________________________________________________________________________________________________________

    # FILTRO 3 --------- filtra por dias da semana a quantidade de chamados
    chamados_total_sem = chamados_total.groupby(['dia_de_abertura', 'n_chamados'])[
        'n_chamados'].sum().sort_values(ascending=False)
    # _____________________________________________________________________________________________________________________

    # Filtro 4 -------- filtra a quantidade de chamados por produto
    chamado_por_produto_percent = dados.groupby(['produto', 'mes_de_abertura'])[
        ['n_chamados']].sum().sort_values('n_chamados', ascending=False)
    # _____________________________________________________________________________________________________________________

    # Filtro 5 -------- filtra o percentual de aberturas de chamados por produto
    chamado_por_produto_percent['prod_percente'] = round(
        (chamado_por_produto_percent['n_chamados'] / chamado_por_produto_percent['n_chamados'].sum()) * 100)
    # _____________________________________________________________________________________________________________________

    # Filtro 6 ------- filtra o produto que mais foi aberto chamado
    chamado_por_produto_max = dados.loc[(
        dados['mes_de_abertura'] == mes_opcao)]
    chamado_por_produto_max = chamado_por_produto_max.groupby(['produto', 'mes_de_abertura'])[
        ['n_chamados']].sum().sort_values('n_chamados', ascending=False).head(3)
    # _____________________________________________________________________________________________________________________

    # Filtro 7 -------- filtra o analista que mais consegui registrar chamados
    qtd_chamados_por_analista = filto_ano_mes.groupby(['mes_de_abertura', 'nome_analista'])[
        ['n_chamados']].sum().sort_values('n_chamados', ascending=False).head(2)
    # _____________________________________________________________________________________________________________________
except NameError:
    st.write('Por favor escolha um arquivo !!!')


##################### Código do Painel Interativo ##################################
try:
    total_de_chamados = dados.loc[(dados['mes_de_fechamento'] == mes_opcao)]
    soma_total_de_chamados = total_de_chamados['n_chamados'].sum()

    total_de_chamados_analista = total_de_chamados.loc[(
        dados['nome_analista'] == opcao_analista)]

    analista = opcao_analista
    analista_mult = analista
    mes = mes_opcao

    soma_total_de_chamados_analista = total_de_chamados_analista['n_chamados'].sum(
    )

    percernt_analista = round(
        soma_total_de_chamados_analista/soma_total_de_chamados * 100)

    total_de_chamados_top10 = filto_ano_mes

    total_de_chamados_top10 = total_de_chamados_top10.groupby('nome_da_empresa')[
        'n_chamados'].sum().reset_index().sort_values('n_chamados', ascending=False)

    total_de_chamados_top10 = total_de_chamados_top10.head(8)

    chamado_por_produto = filto_ano_mes

    chamado_por_produto = chamado_por_produto.groupby('produto')['n_chamados'].sum(
    ).reset_index().sort_values('n_chamados', ascending=False)

    dias_da_semana = filto_ano_mes.groupby('dia_de_abertura')['n_chamados'].sum(
    ).reset_index().sort_values('n_chamados', ascending=False)

    temporal = dados.groupby(['ano_de_abertura', 'produto', 'mes_de_abertura'])[
        'n_chamados'].sum().reset_index().sort_values('n_chamados', ascending=False)

    chamados_por_semana = dados.groupby(
        pd.Grouper(key='data_da_abertura', freq='W'))
    chamados_por_semana = chamados_por_semana[[
        'data_da_abertura', 'n_chamados']].sum('n_chamados')

    chamados_por_dia = dados.groupby(pd.Grouper(
        key='data_da_abertura', freq='B'), dropna=True)
    chamados_por_dia = chamados_por_dia[['data_da_abertura', 'n_chamados']]
except NameError:
    pass


# criar chamado por semanas
# calc_teste = filto_ano_mes
# calc_teste['semanas'] = calc_teste.groupby(pd.Grouper(key='data_da_abertura', freq='W'))
# calc_teste #= filto_ano_mes.groupby(['mes_de_abertura','n_chamados'])['n_chamados'].sum()


# chamados_total_func = dados.groupby(['nome_analista','n_chamados'])['n_chamados'].sum().sort_values('n_chamados',ascending=False)
# chamados_total_func

###########################################  Gráficos   ##################################################
# Cores e tons de vermelho
cor_vermelha = '#ff4b4b'
cor_vermelho_transparente = '#ff7e73'


####################################  Segundo container de gráficos #######################################
try:
    col1, col2, col3 = st.columns([2, 1, 1])

    with col1:
        st.write(f'**Total Registrados no mês de** *{mes}*')
        st.info(f'Total **{soma_total_de_chamados}**', icon="💹")

    with col2:
        st.write(f'**Analista**  *{analista}*')
        st.info(f' Registrou **{soma_total_de_chamados_analista}**', icon='📊')

        with col3:
            st.write('**Registado**')
            st.info(f'Representa    **{percernt_analista}** **%**', icon='🎯')

    st.divider()

    colu1, colu2, = st.columns([0.7, 1])

    with colu1:

        grafico_total_chamados_por_empresa = alt.Chart(total_de_chamados_top10).mark_bar(
            color=cor_vermelha,
            cornerRadiusTopLeft=8,
            cornerRadiusTopRight=8,
        ).encode(
            x='n_chamados',
            y='nome_da_empresa',
            tooltip=['nome_da_empresa', 'n_chamados']
        ).properties(title='Analise das empresas que mais consumiram tempo do colaborador interno no mês', height=450
                     ).configure_axis(grid=False).configure_view(strokeWidth=0)
        st.altair_chart(grafico_total_chamados_por_empresa,
                        use_container_width=True)

    with colu2:

        grafico_de_produtos = alt.Chart(chamado_por_produto).mark_bar(
            color=cor_vermelha,
            cornerRadiusTopLeft=8,
            cornerRadiusTopRight=8,
        ).encode(
            x='produto',
            y='n_chamados',
            tooltip=['produto', 'n_chamados']
        ).properties(title='Lista de produtos que mais registram chamados na base', width=50, height=500
                     ).configure_axis(grid=False).configure_view(strokeWidth=0)
        st.altair_chart(grafico_de_produtos, theme='streamlit',
                        use_container_width=True)
    st.divider()

    coll1, coll2,  = st.columns([1.7, 1])
    with coll1:
        line = alt.Chart(temporal).mark_line(
            color=cor_vermelha,
        ).encode(
            x='produto',
            y='n_chamados:Q',
            color='mes_de_abertura:N',
            tooltip=['n_chamados']
        ).properties(title='Comparação de chamados por produtos', width=100, height=500
                     ).configure_axis(grid=False).configure_view(strokeWidth=0)
        st.altair_chart(line, use_container_width=True)

    with coll2:
        st.write('ranking de chamados por semana')
        grafico_de_dias_da_semana = alt.Chart(dias_da_semana).mark_arc(
            innerRadius=80,
            outerRadius=130,
            padAngle=0.018
            # color=cor_vermelha,
            # cornerRadiusTopLeft=8,
            # cornerRadiusTopRight=8,
        ).encode(
            theta=alt.Theta(field="n_chamados",
                            type="quantitative").stack(True),
            color=alt.Color(field="dia_de_abertura",
                            type="nominal").legend(None),
            # x='dia_de_abertura',
            # y='n_chamados',
            tooltip=['dia_de_abertura', 'n_chamados']
        ).properties(width=90, height=399)

        rotuloName = grafico_de_dias_da_semana.mark_text(
            radius=185,  size=15).encode(text="dia_de_abertura")
        rotuloValor = grafico_de_dias_da_semana.mark_text(
            radius=146,  size=12).encode(text="n_chamados")
        st.altair_chart(grafico_de_dias_da_semana + rotuloName +
                        rotuloValor, use_container_width=True)

    st.divider()

except NameError:
    pass


ModuleNotFoundError: ignored